Restarted aspire (Python 3.8.18)

In [1]:
import aspire
import numpy as np
import pandas as pd
from aspire.operators import RadialCTFFilter
from aspire.source.simulation import Simulation
from aspire.volume import LegacyVolume, Volume
from utils import volsCovarEigenvec
import time
from covar_estimation import im_stack_backward
import torch
# Specify parameters
img_size = 101  # image size in square
num_imgs = 2000  # number of images
dtype = np.float32


rank = 4
c = rank + 1
vols = LegacyVolume(
    L=img_size,
    C=c,
    dtype=dtype,
).generate()
vols -= np.mean(vols,axis=0)
sim = Simulation(
    #unique_filters=[RadialCTFFilter(defocus=d) for d in np.linspace(1.5e4, 2.5e4, 7)],
    n=num_imgs,
    vols=vols,
    dtype=dtype,
    amplitudes=1,
    offsets = 0
)

vectorsGD = torch.tensor(volsCovarEigenvec(vols),requires_grad = False)


2024-04-07 16:35:10,871 INFO [aspire.source.image] Creating Simulation with 2000 images.


In [2]:
%load_ext autoreload
%autoreload 2

from covar_sgd import CovarDataset,Covar,CovarTrainer
cds = CovarDataset(sim,vectorsGD = vectorsGD)
covar = Covar(resolution=img_size,rank=rank)#,vectors=vectorsGD.reshape((rank,img_size,img_size,img_size))/cds.im_norm_factor)
batch_size = 1
learning_rate = 1e-4 
momentum = 0.9
reg = 1e-5
gamma_lr = 0.8
gamma_reg = 1
kwargs_dict = {'max_epochs' : 10, 'lr' : learning_rate,'momentum' : momentum,'optim_type' : 'SGD','reg' : reg,'gamma_lr': gamma_lr,'gamma_reg' : gamma_reg}
#kwargs_dict = {'max_epochs' : 10, 'lr' : 1e-10,'momentum' : momentum,'optim_type' : 'Adam','reg' : reg,'gamma_lr': gamma_lr,'gamma_reg' : gamma_reg}
device = torch.device('cuda:0')
dataloader = torch.utils.data.DataLoader(cds,batch_size = batch_size,shuffle = False)#,collate_fn=dataset_collate)
trainer = CovarTrainer(covar,dataloader,device)

In [3]:

from covar_distributed import trainParallel

trainParallel(covar,cds,num_gpus = 8,batch_size = batch_size,**kwargs_dict)



Batch size is not a multiple of number of GPUs used, increasing batch size to 8


KeyboardInterrupt: 

In [3]:
trainer.train(**kwargs_dict)

Epoch 0 , cost value : 7.71e-01,  cosine sim : 0.82, frobenium norm error : 7.73e-01: 100%|██████████| 2000/2000 [01:02<00:00, 32.12it/s]
Epoch 1 , cost value : 8.69e-01,  cosine sim : 0.83, frobenium norm error : 6.89e-01:   6%|▋         | 130/2000 [00:04<00:51, 36.64it/s]

KeyboardInterrupt: 

In [ ]:
%load_ext autoreload
%autoreload 2
from covar_analyzer import CovarAnalyzer
import torch
c = CovarAnalyzer.load('data/tmp2/results.csv')

#c.plotCosineSim()
#c.plotWeightedCosineSim()
c.plotFroErr()




In [ ]:
torch.norm(cds.vectorsGD[1])
#torch.norm(covar.vectors[1])
a,b,c = np.linalg.svd(covar.vectors.detach().cpu().numpy().reshape((4,-1)),full_matrices=False)
b
torch.norm(cds.vectorsGD.reshape((4,-1)),dim=1)